In [1]:
import whisper



In [2]:
model = whisper.load_model("base")
# audio = whisper.load_audio("datasets/scn.m4a")
result = model.transcribe("datasets/scn.m4a")
print(result["text"])

/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


老婆你要去哪裡吃?


In [3]:
import pyaudio
import wave
import threading
import keyboard
import whisper
# import pyautogui
import argparse

In [7]:

    
class AudioRecorder:
    def __init__(self, chunk=1024, sample_format=pyaudio.paInt16, channels=1, fs=44100, filename="output.wav"):
        self.chunk = chunk
        self.sample_format = sample_format
        self.channels = channels
        self.fs = fs
        self.filename = filename
        self.frames = []
        self.recording = False
        self.model = whisper.load_model("base")
        self.language = "English"
        self.p = pyaudio.PyAudio()

    def toggle_recording(self):
        self.recording = not self.recording
        if self.recording:
            print('Recording')
            self.frames = []  # Clear previous recording frames
            threading.Thread(target=self.record).start()
        else:
            print('Stopped recording')

    def record(self):
        stream = self.p.open(
            format=self.sample_format,
            channels=self.channels,
            rate=self.fs,
            frames_per_buffer=self.chunk,
            input=True
        )
        while self.recording:
            data = stream.read(self.chunk)
            self.frames.append(data)
        stream.stop_stream()
        stream.close()
        self.save_audio()

    def transcribe_recording(self):
        options = {
        "language": self.language, 
        "task": "transcribe"
        }
        result = self.model.transcribe(self.filename, **options)
        return result["text"]

    def save_audio(self):
        with wave.open(self.filename, 'wb') as wf:
            wf.setnchannels(self.channels)
            wf.setsampwidth(self.p.get_sample_size(self.sample_format))
            wf.setframerate(self.fs)
            wf.writeframes(b''.join(self.frames))
        
        transcription = self.transcribe_recording()
        print(f'Transcription: {transcription}')
        print(transcription)

    def set_hotkey(self, hotkey):
        keyboard.add_hotkey(hotkey, self.toggle_recording, suppress=True)
        keyboard.wait('esc')
    
    def set_language(self, language):
        self.language = language
        print("The language is currently set to: ", self.language)


def main():
    parser = argparse.ArgumentParser(description='Audio Recorder and Transcriber')
    parser.add_argument('--hotkey', type=str, default='space', help='Hotkey to toggle recording')
    parser.add_argument('--language', type=str, default='en', help='Language for transcription')
    args = parser.parse_args()

    recorder = AudioRecorder()
    recorder.set_language(args.language)
    # recorder.set_hotkey(args.hotkey)
    recorder.toggle_recording()
    recorder.toggle_recording()
    


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--hotkey HOTKEY] [--language LANGUAGE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9013 --control=9011 --hb=9010 --Session.signature_scheme="hmac-sha256" --Session.key=b"be1aaae4-c239-4d27-a0f0-f6a901d90e53" --shell=9012 --transport="tcp" --iopub=9014 --f=/Users/furyhawk/Library/Jupyter/runtime/kernel-v2-54443S2T20k7DA6b1.json


SystemExit: 2

/opt/homebrew/Caskroom/miniforge/base/envs/whisper/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
